In [1]:
# !unzip output.zip

In [2]:
!pip install spektral

In [3]:
from spektral.layers import GCNConv

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [41]:
import numpy as np
import scipy.sparse as sp
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import Dataset, DisjointLoader, Graph
from spektral.layers import GCSConv, GlobalAvgPool
from spektral.layers.pooling import TopKPool
from spektral.transforms.normalize_adj import NormalizeAdj
import pickle as pkl
import networkx as nx
import pdb

In [31]:
################################################################################
# PARAMETERS
################################################################################
learning_rate = 1e-2  # Learning rate
epochs = 400  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 2  # Batch size


################################################################################
# LOAD DATA
################################################################################
import sys
from spektral.data import Dataset, Graph
# sys.path.append('../lib')
# from data_pre_processing import load_data
# sys.path.remove('../lib')
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import AdjToSpTensor, LayerPreprocess

def load_data(circuit_name, path_to_data="data", normalize=False):
    """Load data."""
    names = ["x", "y", "graph"]
    objects = []
    for i in range(len(names)):
        with open(f"{path_to_data}/{circuit_name}.{names[i]}", "rb") as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding="latin1"))
            else:
                objects.append(pkl.load(f))

    x, y, graph = tuple(objects)

    features = sp.csr_matrix(x).astype(int)
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph)).astype(int)
    labels = np.array(y).reshape((-1,1))

    print(adj.shape)
    print(features.shape)
    return adj, features, labels


class CircuitDataset(Dataset):
    def read(self):
        circuits = []
        circs = ['c6288', 'c17', 'c5315', 'c432', 'c499', 'c880', 'c1355', 'c1908', 'c3540']
        # circs = ['c17']
        for circ in circs:
            A, X, labels = load_data(circ, 'output', normalize="")
            circuits.append(Graph(x=X.toarray(), a=A, y=labels))
            print(f"{circ}: {sum(labels)}, {len(labels)}")
        return circuits

dataset = CircuitDataset(transforms=NormalizeAdj())

# Parameters
F = dataset.n_node_features  # Dimension of node features
n_out = dataset.n_labels  # Dimension of the target

# Train/valid/test split
idxs = np.random.permutation(len(dataset))
split_va, split_te = int(0.8 * len(dataset)), int(0.9 * len(dataset))
idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te])
print(idx_tr, idx_va, idx_te)
dataset_tr = dataset[idx_tr]
dataset_va = dataset[idx_va]
dataset_te = dataset[idx_te]
loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs, node_level=True)
loader_va = DisjointLoader(dataset_va, batch_size=batch_size, node_level=True)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, node_level=True)



(6288, 6288)
(6288, 4)
c6288: [846], 6288
(17, 17)
(17, 4)
c17: [2], 17
(5315, 5315)
(5315, 4)
c5315: [599], 5315
(432, 432)
(432, 4)
c432: [60], 432
(499, 499)
(499, 4)
c499: [50], 499
(880, 880)
(880, 4)
c880: [114], 880
(1355, 1355)
(1355, 4)
c1355: [192], 1355
(1908, 1908)
(1908, 4)
c1908: [257], 1908
(3540, 3540)
(3540, 4)
c3540: [406], 3540
[6 3 5 2 0 1 8] [4] [7]


In [42]:
channels = 16           # Number of channels in the first layer
dropout = 0.5           # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 200            # Number of training epochs
es_patience = 10        # Patience for early stopping
N = 4

# Model definition
X_in = Input(shape=(F, ))
fltr_in = Input((None, ), sparse=True)

dropout_1 = Dropout(dropout)(X_in)
graph_conv_1 = GCNConv(channels,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         use_bias=False)([dropout_1, fltr_in])

dropout_2 = Dropout(dropout)(graph_conv_1)
graph_conv_2 = GCNConv(n_out,
                         activation='softmax',
                         use_bias=False)([dropout_2, fltr_in])

# Build model
model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_2)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

tbCallBack_GCN = tf.keras.callbacks.TensorBoard(
    log_dir='./DetectFaultInCircuit',
)
callback_GCN = [tbCallBack_GCN]

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 4)]          0                                            
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 4)            0           input_11[0][0]                   
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
gcn_conv_10 (GCNConv)           (None, 16)           64          dropout_10[0][0]                 
                                                                 input_12[0][0]             

In [43]:
opt = Adam(lr=learning_rate)
loss_fn = CategoricalCrossentropy()


################################################################################
# FIT MODEL
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs[:-1], training=True)
        loss = loss_fn(target, predictions)
        loss += sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(categorical_accuracy(target, predictions))
    return loss, acc


def evaluate(loader):
    output = []
    step = 0
    while step < loader.steps_per_epoch:
        step += 1
        inputs, target = loader.__next__()
        try:
          pred = model(inputs[:-1], training=False)
        except:
          pdb.set_trace()
        outs = (
            loss_fn(target, pred),
            tf.reduce_mean(categorical_accuracy(target, pred)),
        )
        output.append(outs)
    return np.mean(output, 0)


print("Fitting model")
current_batch = epoch = model_loss = model_acc = 0
best_val_loss = np.inf
best_weights = None
patience = es_patience

for batch in loader_tr:
    outs = train_step(*batch)

    model_loss += outs[0]
    model_acc += outs[1]
    current_batch += 1
    if current_batch == loader_tr.steps_per_epoch:
        model_loss /= loader_tr.steps_per_epoch
        model_acc /= loader_tr.steps_per_epoch
        epoch += 1

        # Compute validation loss and accuracy
        val_loss, val_acc = evaluate(loader_va)
        print(
            "Ep. {} - Loss: {:.2f} - Acc: {:.2f} - Val loss: {:.2f} - Val acc: {:.2f}".format(
                epoch, model_loss, model_acc, val_loss, val_acc
            )
        )

        # Check if loss improved for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience = es_patience
            print("New best val_loss {:.3f}".format(val_loss))
            best_weights = model.get_weights()
        else:
            patience -= 1
            if patience == 0:
                print("Early stopping (best val_loss: {})".format(best_val_loss))
                break
        model_loss = 0
        model_acc = 0
        current_batch = 0

################################################################################
# EVALUATE MODEL
################################################################################
print("Testing model")
model.set_weights(best_weights)  # Load best model
test_loss, test_acc = evaluate(loader_te)
print("Done. Test loss: {:.4f}. Test acc: {:.2f}".format(test_loss, test_acc))


Fitting model
Ep. 1 - Loss: 0.00 - Acc: 1.00 - Val loss: 0.00 - Val acc: 1.00
New best val_loss 0.000
Ep. 2 - Loss: 0.00 - Acc: 1.00 - Val loss: 0.00 - Val acc: 1.00
Ep. 3 - Loss: 0.00 - Acc: 1.00 - Val loss: 0.00 - Val acc: 1.00
Ep. 4 - Loss: 0.00 - Acc: 1.00 - Val loss: 0.00 - Val acc: 1.00
Ep. 5 - Loss: 0.00 - Acc: 1.00 - Val loss: 0.00 - Val acc: 1.00
Ep. 6 - Loss: 0.00 - Acc: 1.00 - Val loss: 0.00 - Val acc: 1.00
Ep. 7 - Loss: 0.00 - Acc: 1.00 - Val loss: 0.00 - Val acc: 1.00
Ep. 8 - Loss: 0.00 - Acc: 1.00 - Val loss: 0.00 - Val acc: 1.00
Ep. 9 - Loss: 0.00 - Acc: 1.00 - Val loss: 0.00 - Val acc: 1.00
Ep. 10 - Loss: 0.00 - Acc: 1.00 - Val loss: 0.00 - Val acc: 1.00
Ep. 11 - Loss: 0.01 - Acc: 1.00 - Val loss: 0.00 - Val acc: 1.00
Early stopping (best val_loss: 0.0)
Testing model
Done. Test loss: 0.0000. Test acc: 1.00


In [32]:
data = dataset[4]

In [24]:
data

Graph(n_nodes=17, n_node_features=4, n_edge_features=None, n_labels=1)

In [40]:
# model.predict([[data.x, data.a]])

In [38]:
data.y.shape

(499, 1)